In [40]:
import pandas as pd
import plotly.express as px
import re
import datetime

from Project.Database import Db

In [41]:
year1_hourly, meta = Db.load_data(hourly=True, meta=True, year=1)

In [42]:
year2_hourly, meta = Db.load_data(hourly=True, meta=True, year=2)

In [43]:
plugs = meta.loc[(meta['Parameter'] == "Status_OnOff") & (meta["Measurement_Location"] == "Kitchen") & (~meta.index.str.contains('Solenoid')) & (~meta.index.str.contains('Sens') & (~meta.index.str.contains('Lights')) & (~meta.index.str.contains('Latent')))].index.tolist()  #

In [44]:
len(plugs), plugs

(13,
 ['Load_StatusPlugLoadBlender',
  'Load_StatusPlugLoadToasterOven',
  'Load_StatusApplianceRangeHood',
  'Load_StatusApplianceCooktop',
  'Load_StatusApplianceOven',
  'Load_StatusPlugLoadSlowCooker',
  'Load_StatusPlugLoadToaster',
  'Load_StatusApplianceDishwasher',
  'Load_StatusPlugLoadHandMixer',
  'Load_StatusPlugLoadCanOpener',
  'Load_StatusPlugLoadCoffeeMaker',
  'Load_StatusRefrigerator',
  'Load_StatusMicrowave'])

In [45]:
consumption = meta.loc[((meta['Parameter'] == "Power_Electrical") | (meta['Parameter'] == "Power_Thermal")) & (meta["Description"].str.contains("power consumption" or "used"))].index.tolist()

In [46]:
status_columns = meta.loc[(lambda self:
                           (self["Parameter"] == "Status_OnOff") &
                           (~ self.index.str.contains("SensHeat")) &
                           (~ self.index.str.contains("Cold")))].index.to_list()

In [47]:
kitchenplugs = year1_hourly[["Timestamp"] + plugs].copy()
kitchenplugs["Status"] = kitchenplugs[plugs].sum(axis=1)
kitchenplugs

,Timestamp,Load_StatusPlugLoadBlender,Load_StatusPlugLoadToasterOven,Load_StatusApplianceRangeHood,Load_StatusApplianceCooktop,Load_StatusApplianceOven,Load_StatusPlugLoadSlowCooker,Load_StatusPlugLoadToaster,Load_StatusApplianceDishwasher,Load_StatusPlugLoadHandMixer,Load_StatusPlugLoadCanOpener,Load_StatusPlugLoadCoffeeMaker,Load_StatusRefrigerator,Load_StatusMicrowave,Status
0,2013-07-01 00:00:00,0,0,0,0,0,0,0,0,0,0,0,1,0,1
1,2013-07-01 01:00:00,0,0,0,0,0,0,0,0,0,0,0,1,0,1
2,2013-07-01 02:00:00,0,0,0,0,0,0,0,0,0,0,0,1,0,1
3,2013-07-01 03:00:00,0,0,0,0,0,0,0,0,0,0,0,1,0,1
4,2013-07-01 04:00:00,0,0,0,0,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8756,2014-06-30 20:00:00,0,0,0,0,0,0,0,1,0,0,0,1,0,2
8757,2014-06-30 21:00:00,0,0,0,0,0,0,0,1,0,0,0,1,0,2
8758,2014-06-30 22:00:00,0,0,0,0,0,0,0,1,0,0,0,1,0,2
8759,2014-06-30 23:00:00,0,0,0,0,0,0,0,0,0,0,0,1,0,1


In [48]:
#Extracts hour of Timestamp.
kitchenplugs["HourOfTimestamp"] = kitchenplugs.Timestamp.dt.hour

kitchenplugs

,Timestamp,Load_StatusPlugLoadBlender,Load_StatusPlugLoadToasterOven,Load_StatusApplianceRangeHood,Load_StatusApplianceCooktop,Load_StatusApplianceOven,Load_StatusPlugLoadSlowCooker,Load_StatusPlugLoadToaster,Load_StatusApplianceDishwasher,Load_StatusPlugLoadHandMixer,Load_StatusPlugLoadCanOpener,Load_StatusPlugLoadCoffeeMaker,Load_StatusRefrigerator,Load_StatusMicrowave,Status,HourOfTimestamp
0,2013-07-01 00:00:00,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
1,2013-07-01 01:00:00,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1
2,2013-07-01 02:00:00,0,0,0,0,0,0,0,0,0,0,0,1,0,1,2
3,2013-07-01 03:00:00,0,0,0,0,0,0,0,0,0,0,0,1,0,1,3
4,2013-07-01 04:00:00,0,0,0,0,0,0,0,0,0,0,0,1,0,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8756,2014-06-30 20:00:00,0,0,0,0,0,0,0,1,0,0,0,1,0,2,20
8757,2014-06-30 21:00:00,0,0,0,0,0,0,0,1,0,0,0,1,0,2,21
8758,2014-06-30 22:00:00,0,0,0,0,0,0,0,1,0,0,0,1,0,2,22
8759,2014-06-30 23:00:00,0,0,0,0,0,0,0,0,0,0,0,1,0,1,23


In [49]:
#Creates a dataframe with counts of "StatusOffOn" = 1, grouped by HourOfTimestamp for a selected attribute in kitchenplugs
count_series = kitchenplugs.groupby(["HourOfTimestamp", "Load_StatusPlugLoadHandMixer"]).size().copy()

new_df = count_series.to_frame(name='size').reset_index()

only_on = new_df.loc[new_df["Load_StatusPlugLoadHandMixer"] == 1]
only_on

,HourOfTimestamp,Load_StatusPlugLoadHandMixer,size
3,2,1,2
20,18,1,309
22,19,1,51


In [50]:
hourON = kitchenplugs.groupby(["HourOfTimestamp"])[plugs].sum()
hourON

,Load_StatusPlugLoadBlender,Load_StatusPlugLoadToasterOven,Load_StatusApplianceRangeHood,Load_StatusApplianceCooktop,Load_StatusApplianceOven,Load_StatusPlugLoadSlowCooker,Load_StatusPlugLoadToaster,Load_StatusApplianceDishwasher,Load_StatusPlugLoadHandMixer,Load_StatusPlugLoadCanOpener,Load_StatusPlugLoadCoffeeMaker,Load_StatusRefrigerator,Load_StatusMicrowave
HourOfTimestamp,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,0,363,0
1,0,0,1,1,0,0,0,0,0,0,0,362,1
2,0,0,2,2,1,0,0,0,2,2,0,359,1
3,0,0,1,1,1,0,0,0,0,0,0,360,0
4,0,0,0,0,1,0,0,2,0,0,0,360,0
5,0,0,0,0,1,0,0,2,0,0,0,360,7
6,256,256,0,0,0,0,256,2,0,0,256,359,234
7,0,0,0,0,0,0,0,0,0,0,13,360,0
8,0,0,101,101,101,52,0,0,0,0,0,361,4


In [51]:
kitchenplugs.groupby(["HourOfTimestamp", 'Load_StatusPlugLoadBlender']).size()

HourOfTimestamp  Load_StatusPlugLoadBlender
0                0                             366
1                0                             366
2                0                             364
3                0                             365
4                0                             365
5                0                             365
6                0                             109
                 1                             256
7                0                             365
8                0                             365
9                0                             264
                 1                             101
10               0                             365
11               0                             365
12               0                             365
13               0                             365
14               0                             365
15               0                             365
16               0                    

In [52]:
#Creates a dataframe with counts of "StatusOffOn" = 1, grouped by HourOfTimestamp for all attributes in kitchenplugs

dataframes = list()
for i in plugs:
    count_series = kitchenplugs.groupby(["HourOfTimestamp", i]).size()

    new_df = count_series.to_frame(name=f'{i} ').reset_index()

    only_on = new_df.loc[new_df[i] == 1].copy()
    only_on.drop(columns=[i], inplace=True)
    dataframes.append(only_on)

hourON = pd.DataFrame(list(range(0, 24)), columns=["HourOfTimestamp"])
for i in dataframes:
    hourON = hourON.merge(i, how="outer", on="HourOfTimestamp").fillna(0).sort_values(by='HourOfTimestamp',
                                                                                      ascending=True)

hourON

,HourOfTimestamp,Load_StatusPlugLoadBlender,Load_StatusPlugLoadToasterOven,Load_StatusApplianceRangeHood,Load_StatusApplianceCooktop,Load_StatusApplianceOven,Load_StatusPlugLoadSlowCooker,Load_StatusPlugLoadToaster,Load_StatusApplianceDishwasher,Load_StatusPlugLoadHandMixer,Load_StatusPlugLoadCanOpener,Load_StatusPlugLoadCoffeeMaker,Load_StatusRefrigerator,Load_StatusMicrowave
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,363,0.0
1,1,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,362,1.0
2,2,0.0,0.0,2.0,2.0,1.0,0.0,0.0,0.0,2.0,2.0,0.0,359,1.0
3,3,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,360,0.0
4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,360,0.0
5,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,360,7.0
6,6,256.0,256.0,0.0,0.0,0.0,0.0,256.0,2.0,0.0,0.0,256.0,359,234.0
7,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,360,0.0
8,8,0.0,0.0,101.0,101.0,101.0,52.0,0.0,0.0,0.0,0.0,0.0,361,4.0
9,9,101.0,101.0,0.0,0.0,101.0,102.0,101.0,0.0,0.0,0.0,101.0,361,98.0


In [56]:
import plotly.graph_objects as go
from Project._02PreliminaryDataAnalysis.energy_status_heatmap import normalise_dataframe

#set up data to heatmap
columns = hourON.columns[1:].tolist()
hourON[columns] = normalise_dataframe(hourON[columns]).copy()
z_data = []
for i in hourON[columns]:
    data_list = hourON[i].round(2).values.tolist()
    z_data.append(data_list)

#Annotate z_values
annotations = go.Annotations()
for n, row in enumerate(z_data):
    for m, val in enumerate(row):
        annotations.append(go.Annotation(text="" if z_data[n][m] == 0 else str(z_data[n][m]),
                                         x=hourON["HourOfTimestamp"][m], y=columns[n], xref='x1', yref='y1',
                                         showarrow=False))

#Create heatmap
fig = go.Figure(data=go.Heatmap(
    z=z_data,
    x=hourON["HourOfTimestamp"],
    y=columns,
    colorscale='Teal')
)

fig.update_layout(margin=dict(
    l=30,
    r=30,
    b=30,
    t=50,),
    font=dict(size=16),
    xaxis_nticks=30,
    annotations=annotations, )

fig.update_xaxes(title="Hour of day")
fig.update_yaxes(title="Appliance", showticklabels=True)
fig.update_layout(template='plotly')
fig.write_html(Db.get_save_file_directory(f"Appliance_on_ratio_heatmap.html"))
fig.show()

/Users/christiannielsen/Documents/Conda_environments/lib/python3.9/site-packages/plotly/graph_objs/_deprecations.py:62: DeprecationWarning:

plotly.graph_objs.Annotations is deprecated.
Please replace it with a list or tuple of instances of the following types
  - plotly.graph_objs.layout.Annotation
  - plotly.graph_objs.layout.scene.Annotation


/Users/christiannielsen/Documents/Conda_environments/lib/python3.9/site-packages/plotly/graph_objs/_deprecations.py:143: DeprecationWarning:

plotly.graph_objs.Annotation is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Annotation
  - plotly.graph_objs.layout.scene.Annotation




In [ ]:
#Extracts hour of Timestamp.
year1_hourly["HourOfTimestamp"] = year1_hourly.Timestamp.dt.hour

#Creates a dataframe with counts of "StatusOffOn" = 1, grouped by HourOfTimestamp for all attributes in year1_hours
hourON_all = year1_hourly.groupby(["HourOfTimestamp"])[status_columns].sum()

#set up data to heatmap
columns = hourON_all.columns[1:].tolist()
hourON_all[columns] = normalise_dataframe(hourON_all[columns])
hourON_all["HourOfTimestamp"] = hourON_all.index
z_data = []
for i in hourON_all[columns]:
    data_list = hourON_all[i].round(2).values.tolist()
    z_data.append(data_list)

#Annotate z_values
annotations = go.Annotations()
for n, row in enumerate(z_data):
    for m, val in enumerate(row):
        annotations.append(go.Annotation(text="" if z_data[n][m] == 0 else str(z_data[n][m]),
                                         x=hourON_all["HourOfTimestamp"][m], y=columns[n], xref='x1', yref='y1',
                                         showarrow=False))

#Create heatmap
fig = go.Figure(data=go.Heatmap(
    z=z_data,
    x=hourON_all["HourOfTimestamp"],
    y=columns,
    colorscale='Teal')
)

fig.update_layout(
    margin=dict(
        l=30,
        r=30,
        b=30,
        t=50, ),
    xaxis_nticks=30,
    annotations=annotations)

fig.update_xaxes(title="Hour of day")
fig.update_yaxes(title="Appliance", showticklabels=True)

fig.show()